<a href="https://colab.research.google.com/github/1804054Miraz/My_Thesis_Work/blob/main/My_thesis_data_formation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

448, 450, 451, 453, 454, 455, 456, 457, 458, 459, 460, 461, 402

In [ ]:
import pandas as pd
import glob
import numpy as np
import tarfile
import gzip
import os
dfs = []
serial=[]

# Specify the path to the .tar.gz file
tar_file_path = '/content/drive/MyDrive/Dataset/eeg_full/co2c1000367.tar.gz'
output_directory = "/content/drive/MyDrive/Dataset/EEG_FULL/Non-alcoholic/367/"
# !mkdir /content/drive/MyDrive/Dataset/EEG_FULL/Non-alcoholic/367

In [ ]:
# Extract the files from the .tar.gz archive
with tarfile.open(tar_file_path, 'r:gz') as tar:
    # List all the files in the archive
    file_list = tar.getnames()
    print(file_list)
    # Initialize a list to store DataFrames
    dfs = []

    # Loop through the files in the archive
    for file_name in file_list:
        if file_name.endswith('.gz'):
            extracted_file = tar.extractfile(file_name)
            decompressed_data = gzip.GzipFile(fileobj=extracted_file)
            serial.append(file_name[-6:-3])
            df = pd.read_csv(decompressed_data, delimiter='\t')
            dfs.append(df)

# Now, data_frames contains DataFrames for all the .rd files in the archive





['co2c1000367', 'co2c1000367/co2c1000367.rd.001.gz', 'co2c1000367/co2c1000367.rd.002.gz', 'co2c1000367/co2c1000367.rd.003.gz', 'co2c1000367/co2c1000367.rd.004.gz', 'co2c1000367/co2c1000367.rd.005.gz', 'co2c1000367/co2c1000367.rd.006.gz', 'co2c1000367/co2c1000367.rd.007.gz', 'co2c1000367/co2c1000367.rd.009.gz', 'co2c1000367/co2c1000367.rd.010.gz', 'co2c1000367/co2c1000367.rd.011.gz', 'co2c1000367/co2c1000367.rd.012.gz', 'co2c1000367/co2c1000367.rd.013.gz', 'co2c1000367/co2c1000367.rd.014.gz', 'co2c1000367/co2c1000367.rd.015.gz', 'co2c1000367/co2c1000367.rd.016.gz', 'co2c1000367/co2c1000367.rd.017.gz', 'co2c1000367/co2c1000367.rd.018.gz', 'co2c1000367/co2c1000367.rd.019.gz', 'co2c1000367/co2c1000367.rd.020.gz', 'co2c1000367/co2c1000367.rd.021.gz', 'co2c1000367/co2c1000367.rd.022.gz', 'co2c1000367/co2c1000367.rd.023.gz', 'co2c1000367/co2c1000367.rd.024.gz', 'co2c1000367/co2c1000367.rd.025.gz', 'co2c1000367/co2c1000367.rd.026.gz', 'co2c1000367/co2c1000367.rd.027.gz', 'co2c1000367/co2c10003

In [ ]:
j=0
dcsv = []
for df in dfs:
 # Access a specific cell by row and column index
  if df.shape[0]>=16450:
    value = df.iloc[2, 0]
    mcond = ""
    for i in value[2:]:
      if(i==' '):
        continue
      if(i == ','):
        break
      # print(i)
      mcond +=i
      # print(mcond)

    fcol=df.columns[0]
    fn = fcol[10:13]
    sid = fcol[5]
    file_name = sid+"_"+fn+"_"+serial[j]+"_"+mcond
    j+=1
    #drop first three rows
    df = df.drop(df.index[:3])
    df = pd.DataFrame(df,columns=None)
    #dataframe to csv and then csv to dataframe
    df.to_csv("data.csv")
    df1 = pd.read_csv("data.csv")
    df1 = pd.DataFrame(df1,columns=None)

    #split first column and make 4 columns
    split_columns = df1[df.columns[0]].str.split(expand=True)
    split_columns.columns = ["trial number", "sensor position", "sample number (0-255)", "sensor value (in micro volts)"]
    df1 = pd.concat([df1, split_columns], axis=1)

    # Drop the original 'Column1'
    df1 = df1.drop(columns=[df.columns[0]])

    #drop the unnamed:0
    df1.drop(['Unnamed: 0'], axis=1, inplace=True)
    # print(df1.head(10))

    df1 = df1[~df1.apply(lambda row: row.astype(str).str.contains('#').any(), axis=1)]

    # Select the 4th column (index 3)
    column_4_data = df1.iloc[:, 3].values

    # Calculate the number of segments
    segment_length = 256
    num_segments = len(column_4_data) // segment_length

    # Create a 2D array 'Sensor' with rows for each segment
    Sensor = np.empty((num_segments, segment_length), dtype=column_4_data.dtype)

    # Split the column data into segments and store them in 'Sensor'
    for i in range(num_segments):
      # print(i)
      # Sensor[i]=sensor_pos[i]
      start_index = i * segment_length
      end_index = (i + 1) * segment_length
      Sensor[i] = column_4_data[start_index:end_index]

    # Now, 'Sensor' contains each segment of 255 values from the 4th column in separate rows

    df2 = pd.DataFrame(Sensor)

    #convert to csv download
    file_name = f"{file_name}.csv"
    file_path = output_directory + file_name
    # df2.to_csv(file_path, index=False)
    # dcsv.append(file_name)

In [ ]:
# Specify the directory path
directory = "/content/drive/MyDrive/Dataset/EEG_FULL/Non-alcoholic/367/"
df1=[]

# Use glob to get a list of all .rd.*.gz files in the directory
rd_gz_files = glob.glob(directory + '*.csv')
# print(rd_gz_files)
# Initialize an empty list to store DataFrames
# Loop through the list of .rd.*.gz files and read them into DataFrames
for file in rd_gz_files:
  dff = pd.read_csv(file, delimiter="\t")
  # serial.append(file[-6:-3])
  df1.append(dff)
# Now, the 'dfs' list contains all DataFrames from the .rd.*.gz files
print(len(df1))

103


In [ ]:
print(len(dfs))
print(len(serial))
# for srl in serial:
#   print(srl)

In [ ]:
# Find list of files. how many files in a directory with specific words or alphabet
# files = os.listdir(directory)

# # Filter files containing '364' in their names
# filtered_files = [file for file in files if '347' in file]
# print(len(filtered_files))

# # Sort the filtered files alphabetically
# sorted_files = sorted(filtered_files)

# # Print the filtered files
# for file in filtered_files:
#     print(file)


In [ ]:
# print(df2)

In [ ]:
# import os
# import glob
# import pandas as pd

# # Set the directory where your data files are located
# data_directory = '/content/drive/MyDrive/Dataset/smni_cmi_train/'

# # Define the type of data you want to read (e.g., "a")
# data_type = "a"

# # Create a list of file paths for the selected data type
# data_files = glob.glob(os.path.join(data_directory, f"{data_type}_*.csv"))
# print(data_files)
# # Initialize an empty list to store DataFrames
# data_frames = []

# # Loop through the files and read them into DataFrames
# for file_path in data_files:
#     df = pd.read_csv(file_path)
#     data_frames.append(df)

# # Now, data_frames contains DataFrames for the selected data type ("a")

# # You can access individual DataFrames like data_frames[0], data_frames[1], etc.
# print(len(data_frames))